<a href="https://colab.research.google.com/github/mnansary/pyF2O/blob/master/colab_gen_unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# colab specific task
*   mount google drive
*   change working directory to git repo
*   TPU check



In [0]:
!pip3 install tensorflow==1.13.1

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


In [0]:
cd /content/gdrive/My\ Drive/PROJECTS/MED/pySKIND

## TPU Check

In [0]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

tf.__version__

# DenseNet Model Training
* Define **Parameters for training**
* Define **FLAGS for DenseNet**

### Data

In [5]:
import sys
sys.path.append('.')

from coreLib.utils import readh5
import numpy as np 
class PARAMS:
    DS_DIR  = '/content/gdrive/My Drive/PROJECTS/MED/Train/' # @param
    BATCH_SIZE      = 128  # @param
    NUM_EPOCHS      = 100  # @param
    X_TRAIN_IDEN    = 'X_train.h5'  # @param
    Y_TRAIN_IDEN    = 'Y_train.h5'  # @param
    X_EVAL_IDEN     = 'X_eval.h5'  # @param
    Y_EVAL_IDEN     = 'Y_eval.h5'  # @param
    MODEL_DIR       = '/content/gdrive/My Drive/PROJECTS/MED/MODEL_DIR/' # @param
    MODEL_NAME      = 'denseNet' # @param

X_train=readh5(os.path.join(PARAMS.DS_DIR,PARAMS.X_TRAIN_IDEN))
Y_train=readh5(os.path.join(PARAMS.DS_DIR,PARAMS.Y_TRAIN_IDEN))
X_eval=readh5(os.path.join(PARAMS.DS_DIR,PARAMS.X_EVAL_IDEN))
Y_eval=readh5(os.path.join(PARAMS.DS_DIR,PARAMS.Y_EVAL_IDEN))

X_train=X_train.astype('float32')/255.0
X_eval=X_eval.astype('float32')/255.0

print(X_train.shape)
print(Y_train.shape)
print(X_eval.shape)
print(Y_eval.shape)


(26624, 64, 64, 3)
(26624, 2)
(1024, 64, 64, 3)
(1024, 2)


### Compile Model

In [6]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from coreLib.models import DenseNet

'''
resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu_address)
tf.contrib.distribute.initialize_tpu_system(resolver)
strategy = tf.contrib.distribute.TPUStrategy(resolver)
with strategy.scope():
  >> code here for tf>1.13.1
'''
MODEL_OBJ=DenseNet(
                    image_dim=64,
                    nb_channels=3,
                    nb_classes=2,
                    nb_dense=3, # @param
                    nb_layers=12,# @param
                    nb_filter=12,# @param 
                    growth_rate=12,# @param 
                    weight_decay=None,# @param
                    dropout_rate=None,# @param
                    bottleneck=False,# @param
                    compression=0.5# @param
                   )

model=MODEL_OBJ.get_model()
model.summary()
model.compile(optimizer=Adam(), 
                loss=categorical_crossentropy,
                metrics=['accuracy'])



Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
MODEL_INPUT (InputLayer)        (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
INITAIAL_CONV2D_LAYER (Conv2D)  (None, 64, 64, 24)   648         MODEL_INPUT[0][0]                
__________________________________________________________________________________________________
BATCH_NORM_BLOCK_1_LAYER_1 (Bat (None, 64, 64, 24)   96          INITAIAL_CONV2D_LAYER[0][0]      
__________________________________________________________________________________________________
RELU_BLOCK_1_LAYER_1 (Activatio (None, 64, 64, 24)   0           BATCH_NORM_BLOCK_1_LAYER_1[0][0] 
_____________________________________

### Train

In [0]:
#from tensorflow.keras.callbacks import ModelCheckpoint   
#checkpointer = ModelCheckpoint(filepath=os.path.join(PARAMS.MODEL_DIR,'{}.h5'.format(PARAMS.MODEL_NAME)), 
                               #verbose=1, 
                               #save_best_only=True)
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

model = tf.contrib.tpu.keras_to_tpu_model(model,
                                          strategy=tf.contrib.tpu.TPUDistributionStrategy(tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))
history=model.fit(X_train,Y_train,
                  validation_data=(X_eval,Y_eval),
                  epochs=PARAMS.NUM_EPOCHS,
                  batch_size=PARAMS.BATCH_SIZE, 
                  verbose=1)
#callbacks=[checkpointer],


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.54.23.18:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 13047164762070634554)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 6597454410549484217)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 8340505189229715248)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1,

### Save Model Weights

In [0]:
model.save_weights(os.path.join(PARAMS.MODEL_DIR,'{}_final.h5'.format(PARAMS.MODEL_NAME)))

### Plot Training Histoty

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('LOSS History')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig(os.path.join(PARAMS.MODEL_DIR,'{}_history.png'.format(PARAMS.MODEL_NAME)))